# HYCO-PhiFlow on Google Colab

This notebook sets up and runs your HYCO-PhiFlow experiments on Google Colab with GPU acceleration.

**Important:** Make sure you've uploaded your HYCO-PhiFlow folder to Google Drive first!

## 1. Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 2. Install Dependencies

In [ ]:
# Install required packages
!pip install -q hydra-core omegaconf phiflow==2.5.3 torch torchvision matplotlib tensorboard

## 3. Setup Project Paths

**Update the `GDRIVE_PROJECT_PATH` below to match where you uploaded your project in Google Drive!**

In [ ]:
import os
import shutil
from pathlib import Path

# UPDATE THIS PATH to your Google Drive location
GDRIVE_PROJECT_PATH = "/content/drive/MyDrive/HYCO-PhiFlow"

# Local fast storage paths
LOCAL_PROJECT = "/content/HYCO-PhiFlow"
LOCAL_CACHE = "/content/cache"
LOCAL_DATA = "/content/data"

# Create local directories
os.makedirs(LOCAL_CACHE, exist_ok=True)
os.makedirs(LOCAL_DATA, exist_ok=True)

print(f"✓ Google Drive project: {GDRIVE_PROJECT_PATH}")
print(f"✓ Local project: {LOCAL_PROJECT}")
print(f"✓ Local cache: {LOCAL_CACHE}")
print(f"✓ Local data: {LOCAL_DATA}")

## 4. Copy Code to Local Storage

Copy the source code and configs to local storage for faster access.

In [ ]:
# Copy essential project files to local storage (faster than Drive)
if os.path.exists(LOCAL_PROJECT):
    shutil.rmtree(LOCAL_PROJECT)

# Copy source code, configs, and run script
shutil.copytree(GDRIVE_PROJECT_PATH, LOCAL_PROJECT, 
                ignore=shutil.ignore_patterns('data', 'cache', 'outputs', 'results', 
                                               '__pycache__', '*.pyc', '.git'))

print("✓ Code copied to local storage")

# Change to project directory
%cd {LOCAL_PROJECT}

## 5. Copy or Generate Data

Choose ONE option:
- **Option A:** Copy existing data from Google Drive (if you already have generated data)
- **Option B:** Generate data fresh on Colab (recommended for first time)

In [ ]:
# Option A: Copy existing data from Google Drive (uncomment if needed)
# DATASET_NAME = "smoke_256"  # Change to your dataset
# source_data = f"{GDRIVE_PROJECT_PATH}/data/{DATASET_NAME}"
# if os.path.exists(source_data):
#     shutil.copytree(source_data, f"{LOCAL_DATA}/{DATASET_NAME}")
#     print(f"✓ Copied {DATASET_NAME} data to local storage")

# Option B: Generate data on Colab (recommended)
print("Data will be generated when you run the experiment")

## 6. Check GPU Availability

In [ ]:
import torch

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️ No GPU detected! Go to Runtime → Change runtime type → Select GPU")

## 7. Generate Data (if needed)

Generate simulation data and cache it to local storage for fast training.

In [ ]:
# Generate data for your experiment
# This will store raw data in LOCAL_DATA and cache in LOCAL_CACHE

!python run.py --config-name=smoke_experiment \
    run_params.mode=[generate] \
    data.data_dir={LOCAL_DATA}/smoke_256 \
    data.cache_dir={LOCAL_CACHE}/smoke_256

## 8. Run Training

Train your model using the local cache for maximum speed.

In [ ]:
# Train with local cache (fast!)
!python run.py --config-name=smoke_experiment \
    run_params.mode=[train] \
    data.data_dir={LOCAL_DATA}/smoke_256 \
    data.cache_dir={LOCAL_CACHE}/smoke_256 \
    trainer_params.epochs=100

## 9. Run Evaluation

In [ ]:
# Evaluate the trained model
!python run.py --config-name=smoke_experiment \
    run_params.mode=[evaluate] \
    data.data_dir={LOCAL_DATA}/smoke_256 \
    data.cache_dir={LOCAL_CACHE}/smoke_256

## 10. Save Results Back to Google Drive

Save trained models and results to Google Drive so they persist after the session ends.

In [ ]:
import shutil
from datetime import datetime

# Create timestamped backup folder
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
backup_dir = f"{GDRIVE_PROJECT_PATH}/colab_results_{timestamp}"
os.makedirs(backup_dir, exist_ok=True)

# Copy results
if os.path.exists("results"):
    shutil.copytree("results", f"{backup_dir}/results")
    print("✓ Saved results to Drive")

# Copy outputs (logs)
if os.path.exists("outputs"):
    shutil.copytree("outputs", f"{backup_dir}/outputs")
    print("✓ Saved outputs to Drive")

# Optionally save cache (if you want to reuse it later)
# Warning: This can be large!
# shutil.copytree(LOCAL_CACHE, f"{backup_dir}/cache")

print(f"\n✓ All results saved to: {backup_dir}")

## 11. Quick Test Run (Optional)

Run a quick test to make sure everything works before starting a long training session.

In [ ]:
# Quick test with fewer epochs
!python run.py --config-name=smoke_quick_test \
    data.data_dir={LOCAL_DATA}/smoke_128 \
    data.cache_dir={LOCAL_CACHE}/smoke_128 \
    trainer_params.epochs=2

## 12. Monitor Training (Optional)

View training metrics and loss curves.

In [ ]:
# If you have tensorboard logs, you can view them
%load_ext tensorboard
%tensorboard --logdir outputs

## Tips for Using Colab

1. **Session Management:**
   - Sessions timeout after ~12 hours of inactivity
   - Save checkpoints frequently
   - Use the save cell (#10) regularly during long training

2. **Speed Optimization:**
   - Always use local storage (`/content/`) for data and cache
   - Only save results back to Drive, not intermediate data
   - Keep raw data and cache in `/content/` during training

3. **Resource Limits:**
   - Free tier: ~12-15GB GPU RAM, ~12GB system RAM
   - Adjust batch size if you get OOM errors
   - Monitor GPU usage: `!nvidia-smi`

4. **Resuming Work:**
   - If session disconnects, re-run setup cells (1-6)
   - Your saved models are safe in Google Drive
   - You may need to regenerate cache if not saved

5. **Alternative Experiments:**
   - Change `--config-name=` to run different experiments
   - Options: `burgers_experiment`, `heat_physical_experiment`, etc.
   - Modify parameters with Hydra overrides as shown above